# Read Libs

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests as req
import os
from bs4 import BeautifulSoup
import time



# display and output settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML




display(HTML('''
<style>
    .container { width:85% !important; }
</style>
'''))

pd.set_option('display.min_rows', 100)

# Read in Tournament Details

In [2]:
path_tournament_detail = "../data/transformed_data/tournament_info/2024-10-06 16h20m45s/tournament_details_transformed.csv"
df_tournament_details = pd.read_csv(path_tournament_detail)

In [10]:
output_path = "../data/raw_data/weather/hourly_weather/"
timestamp_run = dt.datetime.now().strftime('%Y-%m-%d %Hh%Mm%Ss')

os.mkdir(os.path.join(output_path, timestamp_run))

df_unique_cities.to_csv(f'{output_path}/{timestamp_run}/weather_city_base_urls.csv', index=False)

# Get Table from URLs

In [22]:
# Find the table element on the webpage
daily_table_xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table"
table = driver.find_element(By.XPATH, daily_table_xpath)

# Extract the HTML content of the table
table_html = table.get_attribute('outerHTML')

# Use pandas to read the HTML content into a DataFrame
df_hourly_weather = pd.read_html(table_html)[0]


/var/folders/sj/kb7kr7w572z54pgv0pftb9v00000gn/T/ipykernel_55807/1463818549.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_hourly_weather = pd.read_html(table_html)[0]


In [ ]:
# elements for other weather inputs
    
# month_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[1]"
# month_selection = driver.find_element(By.XPATH, month_selection_xpath)

# day_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[2]"
# day_selection = driver.find_element(By.XPATH, day_selection_xpath)

# year_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[3]"
# year_selection = driver.find_element(By.XPATH, year_selection_xpath)

# month_selection.click()
# month_selection.send_keys("January")

# day_selection.click()
# day_selection.send_keys("1")

# year_selection.click()
# year_selection.send_keys("2020")
# year_selection.send_keys(Keys.RETURN)